# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import db_connection_config as config

In [2]:
#conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
conn = psycopg2.connect(config.Config().connection_string(dbname="sparkifydb"))
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
print(song_files)
#print(len(song_files))


['c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json', 'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json', 'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAADZ128F9348C2E.json', 'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAAEF128F4273421.json', 'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAAFD128F92F423A.json', 'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAAMO128F1481E7F.json', 'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAAMQ128F1460CD3.json', 'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_

In [6]:
filepath = song_files[0]
filepath


'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json'

In [7]:
df_array = []
list(map(lambda filepath: df_array.append(pd.read_json(filepath, lines=True)), song_files))
df = pd.concat(df_array)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
0,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:

song_data_dict = df[["song_id", "title", "artist_id", "year", "duration"]].to_dict('records')
song_data_dict

[{'song_id': 'SOMZWCG12A8C13C480',
  'title': "I Didn't Mean To",
  'artist_id': 'ARD7TVE1187B99BFB1',
  'year': 0,
  'duration': 218.93179},
 {'song_id': 'SOCIWDW12A8C13D406',
  'title': 'Soul Deep',
  'artist_id': 'ARMJAGH1187FB546F3',
  'year': 1969,
  'duration': 148.03546},
 {'song_id': 'SOXVLOJ12AB0189215',
  'title': 'Amor De Cabaret',
  'artist_id': 'ARKRRTF1187B9984DA',
  'year': 0,
  'duration': 177.47546},
 {'song_id': 'SONHOTT12A8C13493C',
  'title': 'Something Girls',
  'artist_id': 'AR7G5I41187FB4CE6C',
  'year': 1982,
  'duration': 233.40363},
 {'song_id': 'SOFSOCN12A8C143F5D',
  'title': 'Face the Ashes',
  'artist_id': 'ARXR32B1187FB57099',
  'year': 2007,
  'duration': 209.60608},
 {'song_id': 'SOYMRWW12A6D4FAB14',
  'title': 'The Moon And I (Ordinary Day Album Version)',
  'artist_id': 'ARKFYS91187B98E58F',
  'year': 0,
  'duration': 267.7024},
 {'song_id': 'SOMJBYD12A6D4F8557',
  'title': 'Keepin It Real (Skit)',
  'artist_id': 'ARD0S291187B9B7BF5',
  'year': 0,
  '

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
#cur.execute(song_table_insert, song_data)
list(map(lambda song_data: cur.execute(song_table_insert, song_data), song_data_dict))
conn.commit()

In [10]:
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
artist_column_rename = {"artist_name": "name", "artist_location": "location", "artist_latitude": "latitude", "artist_longitude":"longitude"}
artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].rename(columns = artist_column_rename).to_dict('records')[0]
artist_data


{'artist_id': 'ARD7TVE1187B99BFB1',
 'name': 'Casual',
 'location': 'California - LA',
 'latitude': nan,
 'longitude': nan}

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
log_files = get_files('data/log_data')

In [14]:
filepath = log_files[0]

In [15]:
df_array = []
list(map(lambda filepath: df_array.append(pd.read_json(filepath, lines=True)), log_files))
df = pd.concat(df_array)

df.head()


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540345e+12,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [16]:
df.describe()

,itemInSession,length,registration,sessionId,status,ts
count,8057.000000,6821.000000,7.771000e+03,8057.000000,8057.000000,8.057000e+03
mean,21.196351,247.028101,1.540778e+12,598.153903,202.897232,1.542486e+12
std,23.440324,102.968933,2.651496e+08,285.298026,17.993168,7.002979e+08
min,0.000000,15.855870,1.539909e+12,3.000000,200.000000,1.541106e+12
25%,3.000000,197.328530,1.540558e+12,372.000000,200.000000,1.542023e+12
50%,13.000000,232.907300,1.540829e+12,605.000000,200.000000,1.542467e+12
75%,33.000000,274.102400,1.541020e+12,834.000000,200.000000,1.543064e+12
max,127.000000,2594.873020,1.541098e+12,1114.000000,404.000000,1.543608e+12


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [17]:
df = df[df.page == "NextSong"]
#df = df[(df.page == "NextSong") & (df.firstName == "Kimber")]
df['start_time'] = pd.to_datetime(df['ts'], unit='ms')
df['hour'] = df['start_time'].dt.hour
df['day'] = df['start_time'].dt.day
df['week'] = df['start_time'].dt.isocalendar().week
df['month'] = df['start_time'].dt.month
df['year'] = df['start_time'].dt.year
df['weekday'] = df['start_time'].dt.dayofweek > 4 
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,ts,userAgent,userId,start_time,hour,day,week,month,year,weekday
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,...,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01 21:01:46.796,21,1,44,11,2018,False
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,...,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01 21:05:52.796,21,1,44,11,2018,False
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,...,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01 21:08:16.796,21,1,44,11,2018,False
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,...,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01 21:11:13.796,21,1,44,11,2018,False
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,...,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01 21:17:33.796,21,1,44,11,2018,False


In [18]:
t = df[["start_time", "hour", "day", "week", "month","year", "weekday"]]
count = len(t)


In [19]:
count_distinct = len(t.drop_duplicates())
count_distinct

6813

In [20]:
time_df = t.drop_duplicates().sort_values(by="start_time", ascending=True)
time_df.head()

,start_time,hour,day,week,month,year,weekday
2,2018-11-01 21:01:46.796,21,1,44,11,2018,False
4,2018-11-01 21:05:52.796,21,1,44,11,2018,False
5,2018-11-01 21:08:16.796,21,1,44,11,2018,False
6,2018-11-01 21:11:13.796,21,1,44,11,2018,False
7,2018-11-01 21:17:33.796,21,1,44,11,2018,False


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
for row in time_df.to_dict('records'):    
    cur.execute(time_table_insert, row)
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
user_column_rename = {"userId": "user_id", "firstName": "first_name", "lastName": "last_name"}
user_df = df[["userId", "firstName","lastName", "gender", "level"]].rename(columns = user_column_rename)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [23]:
for row in user_df.to_dict('records'):    
    cur.execute(user_table_insert, row)
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [24]:
df[["artist","song","length"]].head()

,artist,song,length
2,Des'ree,You Gotta Be,246.30812
4,Mr Oizo,Flat 55,144.03873
5,Tamba Trio,Quem Quiser Encontrar O Amor,177.18812
6,The Mars Volta,Eriatarka,380.42077
7,Infected Mushroom,Becoming Insane,440.26730


In [25]:
#select which columns to use
songplay_selected_columns = ["length", "start_time", "userId", "level", "song", "artist", "sessionId", "location", "userAgent"]
#rename columns so it matches the sql params
songplay_column_rename = {"userId":"user_id", "song": "song_title", "artist": "artist_name",  "sessionId": "session_id", "userAgent":"user_agent", "length":"stream_duration"}

#SOMEWHERE I NEED TO APPEND THE SONG AND ARTIST ID TO
#  THE DICTIONARY
#  THE DATAFRAME
songplay_df = df[songplay_selected_columns].rename(columns = songplay_column_rename)
songplay_df["artist_id"] = None
songplay_df["song_id"] = None
songplay_df.head()

,stream_duration,start_time,user_id,level,song_title,artist_name,session_id,location,user_agent,artist_id,song_id
2,246.30812,2018-11-01 21:01:46.796,8,free,You Gotta Be,Des'ree,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None
4,144.03873,2018-11-01 21:05:52.796,8,free,Flat 55,Mr Oizo,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None
5,177.18812,2018-11-01 21:08:16.796,8,free,Quem Quiser Encontrar O Amor,Tamba Trio,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None
6,380.42077,2018-11-01 21:11:13.796,8,free,Eriatarka,The Mars Volta,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None
7,440.26730,2018-11-01 21:17:33.796,8,free,Becoming Insane,Infected Mushroom,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",None,None


In [26]:

for row in songplay_df.to_dict("records"):
    # get songid and artistid from song and artist tables    
    cur.execute(song_select, row)
    results = cur.fetchone()
    if results:                   
        songid, artistid = results
        row["song_id"]= results[0]
        row["artist_id"] = results[1]
        print(row)
    # insert songplay record    
    cur.execute(songplay_table_insert, row)
    conn.commit()

{'stream_duration': 218.93179, 'start_time': Timestamp('2018-11-23 23:08:48.796000'), 'user_id': '47', 'level': 'free', 'song_title': "I Didn't Mean To", 'artist_name': 'Casual', 'session_id': 488, 'location': 'New York-Newark-Jersey City, NY-NJ-PA', 'user_agent': '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"', 'artist_id': 'ARD7TVE1187B99BFB1', 'song_id': 'SOMZWCG12A8C13C480'}


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [27]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.